## Dependencies

In [1]:
import numpy as np
import pandas as pd

import utils, selection, training, constants

## I/O - Initialization

In [2]:
# load the feature dataset as a dataframe
channel_names =  ["Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8", "FT7", "FC3", "FCZ", "FC4", "FT8", "T3", "C3", "Cz", "C4", "T4", "TP7", "CP3", "CPz", "CP4", "TP8", "T5", "P3", "PZ", "P4", "T6", "O1", "Oz" , "O2"]

#channel subsets
#test_channels_1 = ["Cz","CP3","CPz","P3"]
#test_channels_2 = ["Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8", "FT7", "FC3", "FCZ", "FC4", "FT8", "T3", "C3", "Cz", "C4", "T4", "TP7", "CP3", "CPz", "CP4", "TP8", "T5", "P3", "PZ", "P4", "T6", "O1", "Oz" , "O2"]
#test_channels_3 = ["Cz","CP3","CPz","P3","FT7","FC3","FCZ","FC4","C4","CP4","TP8","T5","PZ","T6","O1","Oz","O2"]

dataset = utils.data_loader(constants.MAIN_CSV_FILE)
#reduced_dataset_all = utils.channel_selection(dataset, channel_names)
reduced_dataset_target = utils.channel_selection(dataset, constants.SELECTED_CHANNELS)
#reduced_dataset_1 = utils.channel_selection(dataset, test_channels_1)
#reduced_dataset_1 = utils.channel_selection(dataset, test_channels_1)
#reduced_dataset_2 = utils.channel_selection(dataset, test_channels_2)
#reduced_dataset_3 = utils.channel_selection(dataset, test_channels_3)

#all_features = reduced_dataset_all.columns[:len(reduced_dataset_all.columns) - 1]

## Per-Channel Training+Incremental Training

In [ ]:
#result = incremental_training(dataset=dataset, channel_list=channel_list, feature_subset=all_features, models=['K-NN'], mode='feature', save=True)
#calculate accuracy for each channel
for channel in channel_names:
    print(channel)
    models = ['K-NN', 'GBC']
    dataset = utils.data_loader(csv_file)
    reduced_dataset = utils.channel_selection(dataset, [channel])
    data = training.data_preparation(dataset=reduced_dataset, feature_subset=all_features)
    for model in models:
        model_training(data, model, stats=False, cm=False)

## Chi-Square

In [3]:
feature_subsets = {}

feature_subsets['hjorth_features'] = ['activity', 'mobility', 'complexity']

feature_subsets['stat_features'] = ['skew', 'kurtosis', 'rms', 'dfa', 'mean_abs_sec_dif']

feature_subsets['time_features'] = feature_subsets['hjorth_features'] + feature_subsets['stat_features']

feature_subsets['bands'] = ['delta_power', 'theta_power',
       'alpha_power', 'beta_power', 'gamma_power', 'gamma_beta', 'gamma_alpha',
       'gamma_theta', 'gamma_delta', 'beta_alpha', 'beta_theta', 'beta_delta',
       'alpha_theta', 'alpha_delta', 'theta_delta']

feature_subsets['spec_features'] = ['spc_cnt', 'spc_roff', 'zc', 'slope']

feature_subsets['mfcc_features'] = ['mfcc_0', 'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9']

feature_subsets['mel_features'] = ['mel_0', 'mel_1', 'mel_2', 'mel_3', 'mel_4', 'mel_5'] + ['energy_mel_5','energy_mel_4', 'energy_mel_3', 'energy_mel_2', 'energy_mel_1', 'energy_mel_0']
       
feature_subsets['coeff_features'] = feature_subsets['mfcc_features'] + feature_subsets['mel_features']

feature_subsets['freq_features'] = feature_subsets['spectral_features'] + feature_subsets['coeff_features']

feature_subsets['all'] = feature_subsets['freq_features'] + feature_subsets['time_features']

# 'mel_6', 'mel_7', 'mel_8', 'mel_9', 'mel_10', 'mel_11', 'mel_12', 'mel_13', 'mel_14', 'mel_15', 'mel_16', 'mel_17'
#feature_subsets['chr_features'] = ['chr_0', 'chr_1', 'chr_2', 'chr_3', 'chr_4', 'chr_5', 'chr_6', 'chr_7', 'chr_8', 'chr_9', 'chr_10', 'chr_11', 'chr_12', 'chr_13', 'chr_14', 'chr_15', 'chr_16', 'chr_17', 'chr_18', 'chr_19']
#feature_subsets['ton_features'] = ['ton_0', 'ton_1', 'ton_2', 'ton_3', 'ton_4', 'ton_5']
#feature_subsets['music'] = feature_subsets['chr_features'] + feature_subsets['ton_features']

In [4]:
import numpy as np
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = training.data_preparation(dataset=reduced_dataset_target, feature_subset=feature_subsets['all'])
X = np.concatenate([X_train, X_test])
X = scaler.fit_transform(X)
y = np.concatenate([y_train, y_test]).flatten()

chi2_stats, p_values = chi2(X, y)

In [20]:
chi2_dict = {}
for idx, feature in enumerate(feature_subsets['all']):
    chi2_dict[feature] = chi2_stats[idx]
chi_sorted = dict(sorted(chi2_dict.items(), key=lambda item: item[1], reverse=True))
chi_sorted_list = list(chi_sorted.keys())

In [21]:
print(chi_sorted_list)

['complexity', 'alpha_power', 'energy_mel_4', 'energy_mel_2', 'energy_mel_5', 'spc_roff', 'mel_4', 'energy_mel_3', 'mel_2', 'mel_3', 'energy_mel_1', 'mel_5', 'spc_cnt', 'gamma_beta', 'mel_1', 'theta_power', 'rms', 'beta_alpha', 'gamma_alpha', 'alpha_delta', 'mfcc_8', 'mobility', 'mfcc_2', 'energy_mel_0', 'mel_0', 'mfcc_3', 'mfcc_6', 'alpha_theta', 'mfcc_0', 'gamma_theta', 'delta_power', 'mean_abs_sec_dif', 'zc', 'mfcc_1', 'gamma_delta', 'activity', 'dfa', 'mfcc_5', 'beta_theta', 'theta_delta', 'mfcc_4', 'beta_power', 'mfcc_7', 'gamma_power', 'mfcc_9', 'skew', 'slope', 'beta_delta', 'kurtosis']


In [8]:
chi_sorted

{'complexity': 94.28211862279358,
 'alpha_power': 61.61758765555503,
 'energy_mel_4': 55.966792942887594,
 'energy_mel_2': 50.35524485210654,
 'energy_mel_5': 49.049443325165676,
 'spc_roff': 45.335745653110536,
 'mel_4': 44.463207191022846,
 'energy_mel_3': 44.17877030308956,
 'mel_2': 44.16600415763707,
 'mel_3': 37.11520689875526,
 'energy_mel_1': 37.06779659270134,
 'mel_5': 36.71753654732982,
 'spc_cnt': 32.83364136602729,
 'gamma_beta': 32.70304573869533,
 'mel_1': 31.22485063759271,
 'theta_power': 30.496223978846125,
 'rms': 24.313244174897562,
 'beta_alpha': 23.22056404804195,
 'gamma_alpha': 22.58373946911419,
 'alpha_delta': 20.175210830953596,
 'mfcc_8': 19.043161553563458,
 'mobility': 17.024258058956406,
 'mfcc_2': 16.639397142777554,
 'energy_mel_0': 15.08055575705373,
 'mel_0': 14.893527655872292,
 'mfcc_3': 14.875414600070316,
 'mfcc_6': 11.93753593845356,
 'alpha_theta': 11.095610187638865,
 'mfcc_0': 10.901002678919598,
 'gamma_theta': 9.11242206872141,
 'delta_power

## ANOVA

In [9]:
variance_df, variance_dict = selection.variance_thresholding(reduced_dataset_target, threshold=0.2)

In [11]:
from sklearn.feature_selection import f_classif
scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = training.data_preparation(dataset=reduced_dataset_target, feature_subset=feature_subsets['all'])
X = np.concatenate([X_train, X_test])
X = scaler.fit_transform(X)
y = np.concatenate([y_train, y_test]).flatten()

f_statistic, p_values = f_classif(X, y)

In [17]:
f_dict = {}
for idx, feature in enumerate(feature_subsets['all']):
    f_dict[feature] = f_statistic[idx]
f_sorted = dict(sorted(f_dict.items(), key=lambda item: item[1], reverse=True)) 
anova_sorted_list = list(f_sorted.keys())

In [19]:
print(anova_sorted_list)

['energy_mel_2', 'mel_2', 'mel_3', 'energy_mel_1', 'energy_mel_3', 'energy_mel_4', 'mel_1', 'mel_4', 'spc_roff', 'complexity', 'energy_mel_5', 'mel_5', 'rms', 'spc_cnt', 'alpha_power', 'theta_power', 'mfcc_8', 'mfcc_2', 'gamma_beta', 'mel_0', 'energy_mel_0', 'mfcc_3', 'mobility', 'mfcc_0', 'mfcc_6', 'zc', 'dfa', 'mean_abs_sec_dif', 'beta_alpha', 'alpha_delta', 'mfcc_1', 'gamma_alpha', 'delta_power', 'alpha_theta', 'gamma_theta', 'activity', 'mfcc_5', 'gamma_delta', 'mfcc_4', 'beta_theta', 'skew', 'beta_power', 'theta_delta', 'mfcc_7', 'slope', 'mfcc_9', 'gamma_power', 'beta_delta', 'kurtosis']


In [13]:
f_sorted

{'energy_mel_2': 1716.6963971410814,
 'mel_2': 1683.5683365035482,
 'mel_3': 1270.7767147420525,
 'energy_mel_1': 1270.0349458609455,
 'energy_mel_3': 1265.4877095361082,
 'energy_mel_4': 1256.9155811020325,
 'mel_1': 1243.8409361591785,
 'mel_4': 1234.9127960983778,
 'spc_roff': 1179.7978846086457,
 'complexity': 1171.7471438323403,
 'energy_mel_5': 1117.4046251154236,
 'mel_5': 1112.9821347972538,
 'rms': 724.8313528466713,
 'spc_cnt': 717.5656346396394,
 'alpha_power': 674.0351116859352,
 'theta_power': 659.5643477476857,
 'mfcc_8': 541.3803847829643,
 'mfcc_2': 457.90180619721406,
 'gamma_beta': 427.447151720878,
 'mel_0': 419.34135497630484,
 'energy_mel_0': 388.9095010390218,
 'mfcc_3': 360.7109153467148,
 'mobility': 347.54850066696844,
 'mfcc_0': 332.5889617866217,
 'mfcc_6': 326.9010075256607,
 'zc': 265.2214986783893,
 'dfa': 256.86395209632826,
 'mean_abs_sec_dif': 256.2064104997069,
 'beta_alpha': 249.9364197046969,
 'alpha_delta': 223.31895499403223,
 'mfcc_1': 222.7380161

## Manual Feature Selection

In [5]:
anova_selected = ['spc_roff', 'complexity', 'mel_11', 'mel_10', 'mel_6', 'mel_7', 'mel_12', 'mel_9', 'mel_8', 'mel_14', 'mel_13', 'rms', 'spc_cnt', 'mel_5', 'alpha_power', 'theta_power', 'mfcc_8', 'mfcc_2', 'gamma_beta', 'mel_4', 'mfcc_3', 'mel_3', 'mobility', 'mfcc_0', 'mfcc_6', 'mel_2', 'mel_1', 'zc', 'mean_abs_sec_dif', 'dfa', 'beta_alpha', 'alpha_delta', 'mfcc_1', 'gamma_alpha', 'mel_0', 'delta_power', 'alpha_theta', 'gamma_theta', 'activity', 'mfcc_5', 'gamma_delta', 'mfcc_4', 'beta_theta', 'skew', 'beta_power', 'theta_delta', 'mfcc_7', 'slope']
data = training.data_preparation(dataset=reduced_dataset_target, feature_subset=anova_selected)
for model in constants.ALL_MODELS:
    training.model_training(data, model, stats=True, cm=False)


==== Stats_dict for the K-NN model ====
Training Accuracy:  0.7527047913446677
Test Accuracy:  0.7435105067985167
Sensitivity (Recall): 0.7391304347826086
Precision: 0.74375
F1_score: 0.7414330218068534
AUC: 0.7434889566286967
Logloss: 9.244818390904586


==== Stats_dict for the SVM model ====
Training Accuracy:  1.0
Test Accuracy:  0.927070457354759
Sensitivity (Recall): 0.9465838509316771
Precision: 0.910394265232975
F1_score: 0.9281364190012181
AUC: 0.9271664642112261
Logloss: 2.6286471569319065


==== Stats_dict for the DTC model ====
Training Accuracy:  0.8360123647604327
Test Accuracy:  0.7571075401730532
Sensitivity (Recall): 0.7751552795031056
Precision: 0.7464114832535885
F1_score: 0.7605118829981717
AUC: 0.7571963359385147
Logloss: 8.754731632832534


==== Stats_dict for the RFC model ====
Training Accuracy:  1.0
Test Accuracy:  0.8331273176761433
Sensitivity (Recall): 0.8596273291925466
Precision: 0.8150765606595995
F1_score: 0.8367593712212817
AUC: 0.8332576990366177
Loglo

## P-Value Thresholding

In [45]:
channel_subsets = [channel_names, test_channels_1, test_channels_2, test_channels_3]
subset_names = ['chn_all', 'chn_1', 'chn_2', 'chn_3']
p_val_df = pd.DataFrame(columns=subset_names)

for i in range(len(channel_subsets)):
    dataset = data_loader(csv_file)
    reduced_dataset_i = utils.channel_selection(dataset, channel_subsets[i])
    p_i, p_i_val = selcetion.p_value_thresholding(reduced_dataset_i, feature_subset=all_features)
    p_val_df[subset_names[i]] = p_i_val    

p_val_df.to_csv('outs/p_values_by_channels.csv')

a='''
p_all, p_all_val = p_value_thresholding(reduced_dataset_all, feature_subset=all_features)
    p_1, p_1_val = p_value_thresholding(reduced_dataset_1, feature_subset=all_features)
    p_2, p_2_val = p_value_thresholding(reduced_dataset_2, feature_subset=all_features)
    p_3, p_3_val = p_value_thresholding(reduced_dataset_3, feature_subset=all_features)
'''

/Users/kemalcankucuk/Documents/PAWS Work/eeg-attention/selection.py:77: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat, p_value = stats.ttest_ind(
/Users/kemalcankucuk/Documents/PAWS Work/eeg-attention/selection.py:77: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat, p_value = stats.ttest_ind(
/Users/kemalcankucuk/Documents/PAWS Work/eeg-attention/selection.py:77: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  t_stat, p_value = stats.ttest_ind(
/Users/kemalcankucuk/Documents/PAWS Work/eeg-attention/selection.py:77: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancel

## Yurteri's Method

In [ ]:
p_better = []
for i in range(len(accuracies) - 1):
    delta = accuracies[i+1] - accuracies[i]
    if delta <= 0:
        continue
    else:
        
        p_better.append(p_all[i])

data = data_preparation(selected_channels=selected_channels, selected_labels=selected_labels, feature_subset=p_better)
for model in models:
    training, test = model_training(data, model, stats=False, cm=False, verbose=True)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.linspace(1,len(accuracies),len(accuracies)), accuracies)
plt.legend()
plt.savefig('foo.png', bbox_inches='tight')
plt.grid()
plt.show()
plt.legend(['GBC', 'K-NN', 'SVM', 'DTC', 'NN'])

## ReliefF

In [37]:
from ReliefF import ReliefF
dataset = utils.data_loader(csv_file)
best_channel_list = ['CP3', 'Cz', 'CPz', 'P3']
reduced_dataset = utils.channel_selection(dataset, best_channel_list)
X_train, X_test, y_train, y_test = training.data_preparation(dataset=reduced_dataset, feature_subset=all_features)
X = np.concatenate([X_train, X_test])
y = np.concatenate([y_train, y_test]).flatten()
fs = ReliefF(n_neighbors=1, n_features_to_keep=79)
rf = fs.fit_transform(X, y)

In [64]:
rf_scores = pd.DataFrame(columns=dataset.columns[1:80])
for idx, col in enumerate(rf_scores.columns):
    rf_scores[col] = rf[:][idx]
rf_scores.to_csv('rf scores.csv')